
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/3daf6684a67b6b8b1b36c004e9dedb8947b56ef7/Get%20Machine%20Scorning%20ruGPT3%20model%20output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Machine Scorning ruGPT3 model output

In [1]:
import os

def is_empty(fpath):  
    return not (os.path.isfile(fpath) and os.path.getsize(fpath) > 0)

#@markdown First, let's choose a model:

model = "rukcna"  #@param ["wikihowto", "kish_verses", "products_desc", "rukcna"]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

!echo "Downloaded and extracted {model} model files"

rukcna.tar.gz       100%[===================>] 443.70M  54.2MB/s    in 9.3s    
Downloaded and extracted rukcna model files


Write your inputs to the model here below the `%%writefile input.txt` line:

In [2]:
%%writefile input.txt


Writing input.txt


If you are out of ideas, the following cell will fetch the example input instead in case you have not specified any input:

In [3]:
if is_empty("input.txt"):
  !cp {model}/example_input.txt input.txt

In [4]:
#@markdown Then, you should specify how much takes on each prompt should the model do (how many variants of outputs should we get for each line of input)

takes = 4  #@param {"type": "integer"}



## Install enviroment

Checking GPU card – the command below should work and show around 12-15k of available RAM

In [5]:
!nvidia-smi

Wed Dec 30 07:00:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

!echo "Installed dependencies"

     |████████████████████████████████| 133kB 6.4MB/s 
     |████████████████████████████████| 573kB 8.5MB/s 
     |████████████████████████████████| 133kB 10.6MB/s 
     |████████████████████████████████| 3.7MB 13.3MB/s 
     |████████████████████████████████| 890kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 7.2MB 54.3MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
Installed dependencies


In [7]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

!echo "Downloaded execution script"

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      
Downloaded execution script


## Get results from the model

Now we finally generate batch results from our input. Here you should check the `add_randomness` flag to get different results: otherwise they'll be pretty similar (mostly like in the channel). Again, feel free to change `temperature`, `k` or `p` where appropriate to presumably get even more crappy results.

In [8]:
#@markdown ### Prepare model

import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

with open("input.txt", "r", encoding="utf-8") as intxt:
  prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * 4

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

In [9]:
%%time

#@markdown ### Start the generation!

add_randomness = False #@param {"type": "boolean"}

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
print("Random seed:", rs)

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k=5 \
    --p=0.95 \
    --seed={str(rs)} \
    --temperature=1.0 \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

found = re.findall(re.compile(r"Context >>> ruGPT:\n" + repr(prompt)[1:-1] + r"\n?(.*?)" + ssc, flags=re.DOTALL), responses)
for pt, rs in found:
  printmd('**' + pt + '**')
  printmd(re.sub(lbc, "<br>", rs).strip() + "\n")

Random seed: 42


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Премьер-министр кабинета министров КНДР Пак Бон Чжу по случаю 73-летия основания ТПК 7 октября направил поздравительную телеграмму премьер-министру кабинета министров Ли Су Ёну.<br>Он выразил уверенность в том, что отношения дружбы и сотрудничества между КНДР и Китаем, имеющие давнюю историю и традицию, будут расширяться и развиваться в соответствии с интересами народов двух стран и общими интересами народов двух стран. И сказал, что желает правительству и народу КНДР дальнейшего укрепления и развития отношений дружбы и сотрудничества между двумя странами.<br>-о-


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Премьер-министр России на заседании Госсовета КНДР 12-го числа встретился с президентом Мун Чжэ Ином, и имел беседу.<br>Президент сказал, что в нынешнем году, году 70-летия установления дипломатических отношений между КНДР и РФ, будет более укреплять и развивать двусторонние отношения. И отметил, что будет прилагать усилия к развитию двусторонних отношений, и отметил о том, что в нынешнем году будет более укреплять и развивать двусторонние отношения между КНДР и РФ.<br>Он подчеркнул, что желает корейскому народу больших успехов в борьбе за строительство экономической державы и процветание страны и отметил о том, что будет прилагать все усилия, чтобы в нынешнем году правительство КНДР внесло вклад в обеспечение стабильности на Корейском полуострове и в регионе.<br>Также он отметил, что будет прилагать усилия к укреплению самозащитных оборонных сил и укреплению сотрудничества с международными организациями в целях защиты мира Корейского полуострова и региона.<br>-о-


...

**В Казахстане обрушился мост через Вселенную**

В провинции Северной Пхёнъан провинции Северной Пхёнъан, где находится место катастрофы судна «Пуэбло-2», обрушились мост и площадь перед ним.<br>По сообщениям, в результате чего мост и площадь перед ним оказались затопленными.<br>В связи с этим в комитете провинции Канвон, центральном управлении гидрометеорологической службы и управлении по ликвидации последствий приняли меры по срочному восстановлению моста и площади перед ним.<br>-о-


**В Казахстане обрушился мост через Вселенную**

В последнее время на днях в разных местах Кореи, в том числе в провинции Канвон, в результате чего прервалось движение поездов на протяжении несколько сот км. и прервалось движение судов, в результате чего прервалось движение поездов на протяжении несколько сот км. и прервалось движение поездов на протяжении несколько сот км.


CPU times: user 1.25 s, sys: 211 ms, total: 1.46 s
Wall time: 10min 36s


## Get more results

Feel free to try another set of inputs by re-running the output below!

It is also strongly suggested to turn on `add_randomness` in order to get more diverse results

In [10]:
%%writefile input.txt


Overwriting input.txt


In [11]:
%%time

#@markdown Specify how much takes on each prompt should the model do (how many outputs should we get for each prompt)

takes = 4  #@param {"type": "integer"}

s = ""

with open("input.txt", "r", encoding="utf-8") as intxt:
  prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * 4

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

#@markdown Choose if you want to add randomness (this is a must-check if you want some new results with the same prompts)

add_randomness = True #@param {"type": "boolean"}

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
print("Random seed:", rs)

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k=5 \
    --p=0.95 \
    --seed={str(rs)} \
    --temperature=1.0 \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", repr(model_meta["prompt"])[1:-1])

found = re.findall(re.compile(r"Context >>> ruGPT:\n" + repr(prompt)[1:-1] + r"\n?(.*?)" + ssc, flags=re.DOTALL), responses)
for pt, rs in found:
  printmd('**' + pt + '**')
  printmd(re.sub(lbc, "<br>", rs).strip() + "\n")

Random seed: 2964


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

По случаю Дня Солнца председатель Государственного Совета и Совмина Республики Куба Мигель Марио Диас-Канель Бермудес и его супруга Пунсинпек Кампос в совместном заявлении для печати отметили, что в связи с тем, что в последнее время в Южной Корее в разгаре идет борьба с курением, они будут прилагать активные усилия к решению вопроса об индексации.<br>Также в заявлении для печати отмечается, что будут прилагать активные усилия к решению вопроса о повышении коэффициента трудовой пенсии.<br>В заявлении для печати отмечено, что правительство Республики Куба будет принимать все необходимые меры для защиты и укрепления здоровья народа и достижения стабильности и развития экономики страны, а также прилагать активные усилия к решению вопроса о повышении минимальной зарплаты.<br>-о-


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Пхеньян, 15 декабря. /ЦТАК/ -- По случаю 70-й годовщины основания КНДР президент России Владимир Путин направил поздравительную телеграмму генеральному секретарю ЦК Коммунистической партии Китая товарищу Си Цзиньпину.<br>В телеграмме отмечается следующее:<br>Уважаемый генеральный секретарь товарищ Си Цзиньпин,<br>Пользуясь этим случаем, от всей души желаю Вам и Вашему народу еще больших успехов в ответственной работе для процветания и развития страны.<br>-о-


...

**В Казахстане обрушился мост через Вселенную**

В связи с обрушением моста через Вселенную 1-го мая в уезде Самчжиён провинции Южной Хванхэ Кореи произошел инцидент обрушения моста.<br>В результате происшествия погиб 1 человек.<br>-о-


**В Казахстане обрушился мост через Вселенную**

В связи с тем, что на днях в Корее обрушилось здание Всемирного Совета Мира, 1-го мая в уезде Кандон провинции Южный Пхёнъан произошел взрыв мостового заграждения.<br>На данный момент на месте происшествия находится временный поверенный в делах КНДР в этой стране.<br>Пострадавшие от сильного разрушения моста, которые находятся на грани смерти, находятся в шоковом состоянии.<br>-о-


CPU times: user 1.16 s, sys: 187 ms, total: 1.35 s
Wall time: 10min 21s


Have fun (hopefully)!